In [15]:
%config Completer.use_jedi = False

In [20]:
!pip install selenium webdriver-manager

In [278]:
import requests as req
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import time
import random
import math
import json

print("Hello World")

Hello World


In [43]:
CHROM_DRIVE_PATH = "/workspace/chromedriver/133.0.6943.142/chromedriver.exe"
BASE_URL = "https://m.stock.naver.com/worldstock/home/USA/industry"

In [75]:
service = Service(CHROM_DRIVE_PATH)
driver = webdriver.Chrome(service=service)

actions = ActionChains(driver)

In [76]:
driver.get(BASE_URL)

In [273]:
def get_categoty_list():
    res = driver.page_source
    soup = bs4.BeautifulSoup(res)
    list_items = soup.select("ul.StockCategory_list__IuLu4 > li")
    return list(map(lambda x : x.text, list_items))

def get_stock_list():
    def formatting(row):
        col_list = row.select("td")

        code = col_list[0].select("span")[0].text 

        company_name = col_list[0].select("span")[1].text

        # 현재가, 증감액
        current_price = float(col_list[1].select("span")[0].text.replace(",", ""))
        change_price = float(col_list[1].select("span")[1].select("span")[0].text.replace(",", ""))

        # 증감 비율
        change_rate = float(col_list[2].text[:-1])

        # 시가 총액, 거래량
        market_cap = int(col_list[3].text.replace(",", "") + "000")
        trade_value = col_list[4].text.replace(",", "")
        if(trade_value == "-"):
            trade_value = 0
        trade_value = int(trade_value)

        return {
            "code": code,
            "회사명": company_name,
            "현재가": current_price,
            "증감액": change_price,
            "증감비율": change_rate,
            "시가총액": market_cap,
            "거래량": trade_value,
        }
    
    res = driver.page_source
    soup = bs4.BeautifulSoup(res)

    return list(map(formatting, soup.select("tr.TableComm_tr__QEe9V")))

def get_page_rate():
    res = driver.page_source
    soup = bs4.BeautifulSoup(res)
    rate_num = soup.select(".VMore_link__Tsoh9 span")[1].text.split(" / ")
    return tuple(map(int, rate_num))

In [265]:
def progress_bar(current, total, category, length=20):
    ratio = current / total  # 진행률 계산
    filled_blocks = int(length * ratio)  # ■ 개수
    empty_blocks = length - filled_blocks  # □ 개수

    bar = "■" * filled_blocks + "□" * empty_blocks
    percent = round(ratio * 100, 2)  # 퍼센트 표시 (소수점 2자리)
    
    print(f"[{bar}] {percent}% ({current}/{total}, {category})", end="\r")

In [59]:
categoty_list = get_categoty_list()

In [275]:
# res = []

category_count = len(categoty_list)

for n, category in enumerate(categoty_list):
    if(n < 71):
        continue
        
    progress_bar(n, category_count, category)
    list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[8]/ul/li[{n + 1}]')
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
    )
    list_itme.click()
    
    time.sleep(random.random())
    
    rate = 0, 1
    
    while(True):
        try:
            reate = get_page_rate()
            break
        except:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1 + random.random())
        
    
    for i in range(math.ceil(rate[1] / max(rate[0], 1))):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            more_button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div[2]/div/div/div[10]/div/a')
            actions.move_to_element(more_button).perform()
            more_button.click()

            time.sleep(0.1 + random.random())
        except:
            time.sleep(2 + random.random())
            
    stock_list = get_stock_list()
    
    res.append({
        "카테코리" : category,
        "주식 목록" : stock_list,
    })
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    prev_button = driver.find_element(by=By.XPATH, value='//*[@id="stockContentWrapper"]/div[8]/div/a')
    prev_button.click()
    
    time.sleep(1.5 + random.random())
#     break
#     if(n == 4):
#         break

In [258]:
list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[8]/ul/li[{10}]')
driver.execute_script(
    "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
)
# list_itme.click()

In [279]:
with open("stock_data.json", "w", encoding="utf-8") as file:
    json.dump(res, file, ensure_ascii=False, indent=4)  

In [276]:
res

[{'카테코리': 'IT 서비스 및 컨설팅',
  '주식 목록': [{'code': 'XNET',
    '회사명': '쉰레이 ADR',
    '현재가': 3.84,
    '증감액': 0.94,
    '증감비율': 32.42,
    '시가총액': 243432000,
    '거래량': 46045941},
   {'code': 'GRRR',
    '회사명': '고릴라 테크놀로지 그룹',
    '현재가': 25.6,
    '증감액': 2.99,
    '증감비율': 13.24,
    '시가총액': 472773000,
    '거래량': 2422871},
   {'code': 'NTNX',
    '회사명': '뉴타닉스',
    '현재가': 77.3,
    '증감액': 7.95,
    '증감비율': 11.46,
    '시가총액': 20710853000,
    '거래량': 4825666},
   {'code': 'SNOW',
    '회사명': '스노우플레이크',
    '현재가': 180.72,
    '증감액': 14.53,
    '증감비율': 8.74,
    '시가총액': 59668876000,
    '거래량': 14163646},
   {'code': 'MIGI',
    '회사명': '모손 인프라 스트럭처 그룹',
    '현재가': 0.57,
    '증감액': 0.04,
    '증감비율': 7.55,
    '시가총액': 10663000,
    '거래량': 69935},
   {'code': 'INVE',
    '회사명': '아이덴티브',
    '현재가': 3.87,
    '증감액': 0.26,
    '증감비율': 7.13,
    '시가총액': 92376000,
    '거래량': 69884},
   {'code': 'TDTH',
    '회사명': '트라이던트 디지털 테크 홀딩스 ADR',
    '현재가': 0.62,
    '증감액': 0.03,
    '증감비율': 4.35,
    '시가총액': 40180